In [37]:
import requests
from bs4 import BeautifulSoup
import time
from itertools import groupby
import os
import random
import json
import datetime
import re
import pandas as pd
import unidecode
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import warnings 
warnings.filterwarnings('ignore')

def get_url(url,save_path):
    result=[]
    count=0
    response=requests.get(url)
    soup=BeautifulSoup(response.text,'lxml')
    try:
        pages=int(soup.find('nav',class_='paginator').find_all('a',attrs={'class': lambda e: e.endswith('paginator__btn') if e else False})[-2].text.strip())
    except:
        pages=1
    for page in range(1,pages+1):
#     for page in range(1,2):
        response2=requests.get(f'{url}?das[rent.square]=2&page={page}')
        soup2=BeautifulSoup(response2.text,'lxml') 
        links=soup2.find('section',class_='a-list a-search-list a-list-with-favs').find_all('div',class_='a-card__inc')
        for link in links:
            result.append(f"https://krisha.kz/{link.find('a').get('href')}")
        count+=1
        print(f'Done: {count}'+f'/{pages}',end='\r')
    with open(f'{save_path}/main_links.json','w',encoding='utf-8') as file:
        json.dump(result,file,indent=4,ensure_ascii=False)        

def get_data(path):
    header={}
    save_path='/'.join(path.split('\\')[:-1])
    data = json.load(open(path,'r', encoding='utf-8'))
    result=[]    
    count=0
    for link in data:
        head=[]
        value=[]
        parameters=[]
        response=requests.get(link)
        soup=BeautifulSoup(response.text,'lxml')
        title=soup.find('h1').text.strip()
        price=soup.find('div',class_='offer__price').text.strip()
        headings=soup.find('div',class_='offer__short-description').find_all('div',class_="offer__info-item")
        for heading in headings:
            head.append(heading.find('div').text.strip())
            value.append(heading.find('div',class_='offer__advert-short-info').text.strip())
        
        for i in range(len(head)):
            parameters.append({
                f'{head[i].strip()}':f'{value[i].strip()}'
                 })
        result.append({
            'title':title,            
            'price':price,
            'parameters':parameters,
            'link':link
        }) 
        count+=1
        time.sleep(random.randint(2, 4))
        print(f'Done: {count}'+f'/{len(data)}',end='\r')
        if count%10==0:
            time.sleep(5)
        if len(result)%10==0:
            with open(f'{save_path}\\result.json','w',encoding='utf-8') as file:
                json.dump(result,file,indent=4,ensure_ascii=False)           
    
    with open(f'{save_path}\\result.json','w',encoding='utf-8') as file:
        json.dump(result,file,indent=4,ensure_ascii=False)      
        

def convert_to_excel(path):
    save_path='/'.join(path.split('\\')[:-1])
    data = json.load(open(path,'r', encoding='utf-8'))
    result=pd.json_normalize(data)
    temp_result=pd.DataFrame()
    for i in range(len(result)):
        temp=pd.DataFrame(np.diag(pd.DataFrame(result.parameters[i])),index=pd.DataFrame(result.parameters[i]).columns).T
        temp_result=temp_result.append(temp,ignore_index=True)
    result=pd.concat([result, temp_result], axis=1)
    result['price']=result['price'].apply(lambda x: unidecode.unidecode(x.split('〒')[0]))
    result['address']=result['title'].apply(lambda x: x.split('м²,')[1] if len(x.split('м²,'))==2 else x)
    result['address']=result['address'].apply(lambda x: x.split('—')[0] if len(x.split('—'))==2 else x)
    lll = ['title','address','price','Цена в месяц','Общая площадь, м²', 'Город', 'Местоположение',
    'Телефон','Безопасность', 'Интернет', 'Название торгового центра, рынка','Что именно?','link']
    col = list(result.columns)
    ttt = []
    for i in lll:
        if i in col: ttt.append(i)
    result = result[ttt]
    result.to_excel(f'{save_path}\\data.xlsx',index=False) 
    
def find_coordinates(path):
    save_path='/'.join(path.split('\\')[:-1])
    data=pd.read_excel(path)
    result=[]
    driver = webdriver.Chrome(executable_path=r'C:\Users\svnduw\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe')
    driver.get('https://yandex.kz/maps/162/almaty/')
    count=0
    for index,row in data.iterrows():
        searchBox=driver.find_element_by_class_name('input__control')    
        searchBox.send_keys(str(row['address']))   
        searchBox.send_keys(Keys.ENTER) 
        time.sleep(3) 
        y=driver.current_url
        y_lon=y.split('?ll=')[1].split('%2C')[0]
        y_lat=y.split('?ll=')[1].split('%2C')[1].split('&')[0]
        data.at[index,'lat']=y_lat
        data.at[index,'lon']=y_lon        
        for m in range(len(row['address'])+5):
                searchBox.send_keys(Keys.BACK_SPACE)
        count+=1
        print(f'Done: {count}'+f'/{len(data)}',end='\r')
        if count%10==0:
            data.to_excel(f'{save_path}\\result.xlsx',index=False)
    data.to_excel(f'{save_path}\\result.xlsx',index=False)
        
    

def main():
    city='almaty'
    type_arenda='magazina' #magazina,kvartiry
    url=f'https://krisha.kz/arenda/{type_arenda}/{city}/'
    save_path=r'C:\Users\svnduw\Desktop\Bekbol\Bekbol project\Новая папка\parsing'
#     get_url(url,save_path)
#     get_data(f'{save_path}\\main_links.json')
#     convert_to_excel(f'{save_path}\\result.json')
    find_coordinates(f'{save_path}\\data.xlsx')
    
if __name__=="__main__":
     main() 